In [1]:
from pyldl.algorithms import LDL_LRR
from pyldl.utils import LDLEarlyStopping
from pyldl.algorithms.base import BaseGD, BaseDeepLDL

In [2]:
import pyldl.applications.facial_emotion_recognition as fer

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
import keras
import tensorflow as tf
import numpy as np
from keras.optimizers import SGD

In [5]:
X, D = fer.load_jaffe('./jaffe/')

In [6]:
temp, test = train_test_split(np.arange(X.shape[0]), test_size=0.1, random_state=0)
X_train, X_val, D_train, D_val = train_test_split(X[temp], D[temp], test_size=0.1, random_state=0)

In [7]:
class LRR(BaseGD, BaseDeepLDL):
    def __init__(self, alpha=1e-2, **kwargs):
        super().__init__(**kwargs)
        self._alpha = alpha
    
    @staticmethod
    def loss_function(D, D_pred):
        return tf.math.reduce_mean(keras.losses.kl_divergence(D, D_pred))

    def _loss(self, X, D, start, end):
        D_pred = self._call(X)
        kld = self.loss_function(D, D_pred)
        rnk = LDL_LRR.ranking_loss(D_pred, self._P[start:end], self._W[start:end]) / (2 * X.shape[0])
        return kld + self._alpha * rnk

    def _before_train(self):
        self._P, self._W = LDL_LRR.preprocessing(self._D)

In [8]:
lrr = LRR()

In [9]:
inputs = keras.Input(shape=X.shape[1:])
encodes = keras.applications.ResNet50(include_top=False, weights='imagenet')(inputs)
pooling = keras.layers.GlobalAveragePooling2D()(encodes)
outputs = keras.layers.Dense(D.shape[1], activation='softmax')(pooling)

In [10]:
model = keras.Model(inputs=inputs, outputs=outputs)

In [11]:
lrr.fit(X_train, D_train, verbose=1, batch_size=32, optimizer=SGD(2e-3), model=model,
        X_val=X_val, D_val=D_val, callbacks=[LDLEarlyStopping(monitor='kl_divergence', patience=None)]);

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1763s 2s/step - loss: 0.0065 - kl_divergence: 0.0265


In [12]:
scores = lrr.score(X[test], D[test], return_dict=True)
for (k, v) in scores.items():
    print(f"{k}: {v:.4f}")

chebyshev: 0.0680
clark: 0.2625
canberra: 0.5416
kl_divergence: 0.0286
cosine: 0.9729
intersection: 0.9099


In [13]:
# Windows
# fer.extract_ck_plus('./CK+/', './CK+/features/', './OpenFace_2.2.0_win_x64/')

In [14]:
X, labels, fps, aus = fer.load_ck_plus('./CK+/', './CK+/features/')

In [15]:
L = keras.utils.to_categorical(labels)

In [16]:
temp, test = train_test_split(np.arange(X.shape[0]), test_size=0.1, random_state=0)
X_train, X_val, L_train, L_val = train_test_split(X[temp], L[temp], test_size=0.1, random_state=0)

In [17]:
alsg = fer.LDL_ALSG()

In [18]:
alsg.fit(X_train, L_train, fps, aus, verbose=1, batch_size=32, optimizer=SGD(1e-4),
         X_val=X_val, L_val=L_val, callbacks=[LDLEarlyStopping(monitor='loss', patience=None)]);

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2410s 2s/step - loss: 7.7337


In [19]:
scores = alsg.score(X[test], L[test], return_dict=True)
for (k, v) in scores.items():
    print(f"{k}: {v:.4f}")

accuracy: 0.9032
